### Задача 2
---
В пункте В2 сталь поступает на условный производственный комплекс, состоящий из сталелитейного и станкостроительного заводов. Он функционирует в течение 5 лет.

Начальный запас стали составляет 3100 т (для решения; интервал для исследования — 1000–10000 т (\*)).
Исходные производственные мощности заводов соответственно:
- по стали — 4300 т в год;
- по станкам — 1300 станков в год (для решения; интервал для исследования — 100–5000 станков). (\*\*)

Сталь расходуется на производство станков (1.1 т на каждый) и собственно стали, а также на расширение производственных мощностей комплекса. При этом каждая тонна стали, направленная на ее производство, обеспечивает выпуск 3.4 т. Тонна стали, идущая на расширение производственной мощности сталелитейного завода, увеличивает последнюю на 0.1 т, а для увеличения производственных мощностей станкостроительного завода на один станок необходимо затратить 15 т стали. Реализация решения о распределении стали на следующий год осуществляется в конце очередного года планируемого периода; станкостроительный завод не может получать более половины имеющегося запаса стали.

Требуется:
1. определить план распределения стали между сталелитейным и станкостроительным заводами, обеспечивающий выпуск максимального количества станков за плановый период;
1. провести исследование (поочередно, по одному фактору) влияния начальных условий (\*) и (\*\*) на структуру и результат решения (т.е. на распределение стали и на объем выпуска станков).


In [1]:
import numpy
from cvxopt.modeling import variable
from cvxopt.modeling import op
import pandas as pd

In [2]:
years = 5 # период
initial_steel_reserve = 6900 # начальный запас стали
production_capacity_steel = 2400 # производственная мощность литейки
production_capacity_machine = 600 # производственная мощность станков

# производство
steel_cost = 1 # потрачено на сталь
steel_rise = 2.7 # получено стали
machine_cost = 1.1 # потрачено на станок
machine_rise = 1 # получено станков

#pc - production capacity
# увеличение производственной мощности
steel_pc_cost = 1 # стоимость прироста мощности литейки
steel_pc_rise = 0.3 # получен прирост мощности литейки
machine_pc_cost = 5 # стоимость прироста мощности литейки
machine_pc_rise = 1 # получен прирост мощности литейки

In [3]:
spent_steel_prod = variable(years, 'на произв. стали') # потрачено на производство стали
spent_machine_prod = variable(years, 'на произв. станков') # потрачено на производство станков
spent_steel_inc = variable(years, 'на ув.кол. стали') # потрачено на увеличение стали
spent_machine_inc = variable(years, 'на ув.кол. станков') # потрачено на увеличение станков

In [4]:
# Получить сталь, которая будет выпущена за год (с коэффициентом)
get_released_steel = lambda year: spent_steel_prod[year] / steel_cost * steel_rise

# Получить станки, выпущенные за указаный год
get_released_machines = lambda year: spent_machine_prod[year] / machine_cost * machine_rise

# Получить запас стали на текущий год
def get_cur_steel_reserve(year):
    if year == 0:
        return initial_steel_reserve
    return get_released_steel(year-1)

# Получить производственную мощь литейки в текущем году
def get_current_steel_production_capacity(year):
    if year == 0:
        return production_capacity_steel
    return production_capacity_steel + sum(spent_steel_inc[:year]) / steel_pc_cost * steel_pc_rise

# Получить производственную мощь по станкам в текущем году
def get_current_machines_production_capacity(year):
    if year == 0:
        return production_capacity_machine
    return production_capacity_machine + sum(spent_machine_inc[:year]) / machine_pc_cost * machine_pc_rise

In [16]:
# Сумма затрат стали в году равна количеству в резерве
d0 = [spent_steel_prod[year] + spent_machine_prod[year] + spent_steel_inc[year] + spent_machine_inc[year] == get_cur_steel_reserve(year) for year in range(years)]

# Будет выпущено стали не более чем возможно
d1 = [get_released_steel(year) <= get_current_steel_production_capacity(year) for year in range(years)]

# Будет выпущено станков не более чем возможно
d2 = [get_released_machines(year) <= get_current_machines_production_capacity(year) for year in range(years)]

# Cтанкостроительный завод не может получать более половины имеющегося запаса стали
d3 = [spent_machine_prod[year] + spent_machine_inc[year] <= 0.5 * get_cur_steel_reserve(year) for year in range(years)]

# И все затраты не могут быть отрицательными
d4 = [spent_steel_prod[year] >= 0 for year in range(years)]
d5 = [spent_machine_prod[year] >= 0 for year in range(years)]
d6 = [spent_steel_inc[year] >= 0 for year in range(years)]
d7 = [spent_machine_inc[year] >= 0 for year in range(years)]

In [17]:
# Решение
problem = op(-sum(spent_machine_prod/machine_cost), d0+d1+d2+d3+d4+d5+d6+d7)  
problem.solve(solver = 'glpk')
problem.status

In [20]:
# Максимальное количество станков
print(-problem.objective.value()[0])

5338.269253872055


In [10]:
# Вывод оптимального решения
humanize = lambda x: [x.value[i] for i in range(years)]
pd.DataFrame({spent_steel_prod.name: humanize(spent_steel_prod),
              spent_machine_prod.name: humanize(spent_machine_prod),
              spent_steel_inc.name: humanize(spent_steel_inc),
              spent_machine_inc.name: humanize(spent_machine_inc),
             })

,на произв. стали,на произв. станков,на ув.кол. стали,на ув.кол. станков
0,888.888889,660.000000,2561.111111,2790.000000
1,1173.456790,1200.000000,26.543210,0.000000
2,1176.406036,1273.800000,407.760631,310.366667
3,1034.233713,1342.080667,553.914435,246.067481
4,1283.258821,1396.215513,112.956692,0.000000
